# First look

* [Priority Population/DAC](https://dot.ca.gov/programs/rail-and-mass-transportation/priority-populations-and-disadvantaged-communities)

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from calitp import *
from calitp.storage import get_fs
from shared_utils import geography_utils, utils

fs = get_fs()
import os

# Formatting the nb
pd.options.display.max_columns = 100
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.options.display.float_format = "{:.2f}".format

# Scripts
import A1_data_prep as data_prep

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Loading in the script

In [2]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/lctop/"
FILE_NAME = "LCTOP_allyears.xlsx"

In [3]:
#Original df 
df1 = to_snakecase(
    pd.read_excel(f"{GCS_FILE_PATH}{FILE_NAME}", sheet_name="LCTOP_Projects")
)

In [4]:
#Cleaned df
df2 = data_prep.clean_lctop()

/home/jovyan/data-analyses/lctop/A1_data_prep.py:74: FutureWarning: The default value of regex will change from True to False in a future version.


In [5]:
district_summary, gdf_dist = data_prep.district_fy_summary()

/home/jovyan/data-analyses/lctop/A1_data_prep.py:74: FutureWarning: The default value of regex will change from True to False in a future version.
/home/jovyan/data-analyses/lctop/A1_data_prep.py:88: FutureWarning: The default value of regex will change from True to False in a future version.
/opt/conda/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


In [39]:
#df2['lead_agency'].sort_values().unique()

In [38]:
df2['lead_agency'].nunique()

175

## Checking out the raw data

In [7]:
df1.shape

(857, 74)

In [8]:
# Get percentages of how many null values per column
df1.isnull().sum() * 100 / len(df1)

count                                              0.70
#                                                  0.70
funding_year                                       0.70
_d                                                 0.70
distr_                                             0.70
project_id#                                        0.70
lead_agency                                        0.70
project_name                                       0.70
project_type                                       0.58
project_sub_type                                  27.42
project_sub_type_ii                                0.70
#2                                                99.53
project_description__short_                       11.79
project_location                                  28.12
start_date                                        12.02
completion_date                                   27.30
contact_name                                       1.05
contact_phone_#                                 

In [9]:
# Check out that values are what I expect for certain columns
value_count_cols = [
    "project_type",
    "lead_agency",
    "distr_",
    "project_name",
    "project_id#",
    "project_sub_type_ii",
]

In [10]:
# https://stackoverflow.com/questions/32589829/how-to-get-value-counts-for-multiple-columns-at-once-in-pandas-dataframe
"""
for column in df1[value_count_cols]:
    print("\n" + column)
    print(df1[column].value_counts())
    
"""

'\nfor column in df1[value_count_cols]:\n    print("\n" + column)\n    print(df1[column].value_counts())\n    \n'

In [11]:
# Find nunique values
for column in df1[value_count_cols]:
    print("\n" + column)
    print(df1[column].nunique())


project_type
3

lead_agency
213

distr_
12

project_name
714

project_id#
851

project_sub_type_ii
12


### Double checking monetary columns


In [13]:
monetary_cols = [
    "puc_99313_funds",
    "puc_99314_funds",
    "total_project_request_99314_+_99313",
    "total_lctop_funds",
    "total_cci_funds",
    "total_project_cost",
    "lctop_%_of_total_project_funds",
]

In [14]:
subset = df1[monetary_cols].fillna(0)

In [15]:
# https://stackoverflow.com/questions/42405572/how-to-compare-two-columns-of-the-same-dataframe
# Compare if PUC 99313 + PUC 99314 equal to total LCTOP Fund
subset["total_project_comp_to_total_LCTOP_funds"] = np.where(
    subset["total_project_request_99314_+_99313"] == subset["total_lctop_funds"],
    "same",
    "different",
)

In [16]:
# Comapre if LCTOP equals CCI
subset["total_lctop_cci"] = np.where(
    subset["total_lctop_funds"] == subset["total_cci_funds"], "same", "different"
)

In [17]:
# Calculate out total_project_request_99314_+_99313 and see if the results equal what was provided to me.
subset["99314_and_99313"] = subset["puc_99313_funds"] + subset["puc_99314_funds"]
subset["my_calc_vs_og_cal"] = np.where(
    subset["99314_and_99313"] == subset["total_project_request_99314_+_99313"],
    "same",
    "different",
)

In [18]:
# Compare total_project_request_99314_+_99313 with total CCI funds
subset["CCI_99313_99314"] = np.where(
    subset["total_project_request_99314_+_99313"] == subset["total_cci_funds"],
    "same",
    "different",
)

In [19]:
# Find value_counts
value_count_cols_2 = [
    "total_project_comp_to_total_LCTOP_funds",
    "total_lctop_cci",
    "my_calc_vs_og_cal",
    "CCI_99313_99314",
]
for column in subset[value_count_cols_2]:
    print("\n" + column)
    print(subset[column].value_counts())


total_project_comp_to_total_LCTOP_funds
different    435
same         422
Name: total_project_comp_to_total_LCTOP_funds, dtype: int64

total_lctop_cci
same         813
different     44
Name: total_lctop_cci, dtype: int64

my_calc_vs_og_cal
same         856
different      1
Name: my_calc_vs_og_cal, dtype: int64

CCI_99313_99314
different    446
same         411
Name: CCI_99313_99314, dtype: int64


In [20]:
# subset.loc[subset['my_calc_vs_og_cal'] == 'different']

### Other steps
* Double check that agencies are only spelled in ONE way, not in multiple ways.
* Change any yes/no open/close columns to lower and strip spaces, so there aren't diffrent options.
* Removed county.
* Make sure all cols are the same data type

In [21]:
#Check values manually 
#df2['lead_agency'].sort_values().unique()

## Initial insights

### How are projects benefitting DAC, are they finished or in progress, how many null values are present, etc?
* [Disadvantaged Communities](https://dot.ca.gov/programs/rail-and-mass-transportation/priority-populations-and-disadvantaged-communities)
* Most projects benefit DAC.
* Most agencies have a service area with DAC.

In [22]:
# count
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_dict.html
result = {}
for i in df2[data_prep.boolean_cols]:
    result[i] = df2[i].value_counts()

In [23]:
result_df = (
    pd.DataFrame.from_dict(result).fillna(0).rename(columns={"index": "Boolean Values"})
)

result_df = result_df.div(result_df.sum(axis=0), axis=1)

In [24]:
result_df.round(2).style.background_gradient()

,agency_service_area_has_a_dac,does_project_benefit_an_ab_1550_dac,status,qualifying_1_2_mile_low_income_buffer_,ab_1550_low_income_community__household
0,0.000000,0.000000,0.000000,0.180000,0.000000
1,0.000000,0.000000,0.000000,0.190000,0.000000
None,0.000000,0.000000,0.370000,0.480000,0.410000
closed,0.000000,0.000000,0.020000,0.000000,0.000000
no,0.300000,0.340000,0.000000,0.150000,0.050000
open,0.000000,0.000000,0.610000,0.000000,0.000000
yes,0.700000,0.660000,0.000000,0.000000,0.540000


### Summary Tables

In [25]:
sum_cols = [
    "funds_to_benefit_dac",
    "total_project_request_99314_+_99313",
    "total_project_cost",
    "vmt_reduction",
    "ghg_reduction__mtco2e_",
    "diesel_pm_reductions__lbs_",
    "nox_reductions__lbs_",
    "pm_2_5_reductions__lbs_",
    "reactive_organic_gas_reduction__lbs_",
    "fossil_fuel_use_reduction__transportation_",
    "ridership_increase",
    "fossil_fuel_use_reduction__energy_",
    "renewable_energy_generation__kwh_",
]
nunique_cols = ["project_id#", "lead_agency"]

In [26]:
funding_year_summary = geography_utils.aggregate_by_geography(
    df2, group_cols=["funding_year"], nunique_cols=nunique_cols, sum_cols=sum_cols
)

funding_year_summary = (
    data_prep.cols_cleanup(funding_year_summary)
    .sort_values("Funding Year")
    .rename(columns={"Lead Agency": "# of Agencies", "Project Id#": "# of Projects"})
)

/home/jovyan/data-analyses/lctop/A1_data_prep.py:88: FutureWarning: The default value of regex will change from True to False in a future version.


In [27]:
funding_year_summary

,Funding Year,Diesel Pm Reductions Lbs,Fossil Fuel Use Reduction Energy,Fossil Fuel Use Reduction Transportation,Funds To Benefit Dac,Ghg Reduction Mtco2E,Nox Reductions Lbs,Pm 2 5 Reductions Lbs,Reactive Organic Gas Reduction Lbs,Renewable Energy Generation Kwh,Ridership Increase,Total Project Cost,Total Project Request 99314 + 99313,Vmt Reduction,# of Agencies,# of Projects
0,14-15,0.00,0.00,0.00,22753617.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,24165593.00,0.00,83,95
1,15-16,0.00,0.00,0.00,69453304.00,10734.81,0.00,0.00,0.00,0.00,236838.00,532800958.42,74700760.00,1193035.95,107,132
2,16-17,0.00,0.00,0.00,29652105.00,4161658.31,0.00,0.00,0.00,0.00,726533623.50,2625839228.00,34539105.00,10181040666.00,105,126
3,17-18,72745.82,3110295.00,0.00,66033836.71,3615714.54,17961394.95,607338.50,1143040.39,0.00,1728538312.51,2526653147.52,96864564.50,7682500431.24,109,152
4,18-19,343172.65,0.00,74167587.23,91638903.30,3062602.71,11417960.94,469422.60,1476528.45,0.00,709833819.75,4137512637.19,146949406.00,4593429702.34,120,180
5,19-20,1428119.23,-576698452.72,86239774.74,79344810.00,2431934.00,11556745.30,1408618.81,4032264.90,3360000.00,205879775.19,2889873923.16,146054354.00,3096126163.97,110,166


In [28]:
project_type_summary = geography_utils.aggregate_by_geography(
    df2,
    group_cols=["project_sub_type_ii"],
    nunique_cols=nunique_cols,
    sum_cols=sum_cols,
)

project_type_summary = (
    data_prep.cols_cleanup(project_type_summary)
    .sort_values("Lead Agency")
    .rename(columns={"Lead Agency": "# of Agencies", "Project Id#": "# of Projects"})
)

/home/jovyan/data-analyses/lctop/A1_data_prep.py:88: FutureWarning: The default value of regex will change from True to False in a future version.


In [29]:
project_type_summary['# of Projects'].sum()

851

In [30]:
project_type_summary['Funds To Benefit Dac'].sum()

358876576.01

In [31]:
district_summary['Funds To Benefit Dac'].sum()

358876576.01

In [32]:
district_summary['# of Projects'].sum()

851

## Lat Lon

In [33]:
# Get only values w/ lon and lat
df3 = df2.loc[df2["project_location"] != "None"]

In [34]:
df3["project_location_2"] = df1["project_location"].str.split(";").str[0]

/tmp/ipykernel_1204/3693640767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [35]:
df3[["lon", "lat"]] = (
    df3["project_location_2"]
    .str.split(",", 1, expand=True)
    .apply(pd.to_numeric, errors="coerce")
    .fillna(0)
)

/tmp/ipykernel_1204/1674458213.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_1204/1674458213.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [36]:
df3 = df3[(df3["lon"] != 0) & (df3["lat"] != 0)]
len(df3)

371